Load the file

In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
!pip install --user -U nltk
import nltk
nltk.download('punkt')

Requirement already up-to-date: nltk in /root/.local/lib/python3.7/site-packages (3.5)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
import pandas
import csv
import numpy as np # linear algebra
import pandas as pd

sst_home='drive/MyDrive/Colab Notebooks/'
#modify this path 
path=sst_home+'fake-news/train'

news = pd.read_csv(path)
#news = pandas.read_csv(path, sep=',', quoting=csv.QUOTE_NONE, names=["label", "text"])
#shows the first messages
news.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


Cleaning

In [ ]:
import nltk

nltk.download('stopwords')
nltk.download('punkt')

from nltk.corpus import stopwords
stopwords_en = stopwords.words("english")

from nltk import word_tokenize
from nltk.stem.porter import PorterStemmer
import re


def cleanText(text):
    text=str(text).lower()
    
    #tokeniza the text
    tokens=word_tokenize(text)


    
    #remove the stopwords
    tokens = [word for word in tokens if word not in stopwords_en]

    
    
    #(4) obtain the stems
    tokens = [PorterStemmer().stem(word) for word in tokens]

    
    #(5) finally, remove words with len <3 and words that contain numbers, puntuaction, ect
    min_length = 3
    p = re.compile('[a-zA-Z]+');
    filtered_tokens=[]
    for token in tokens:
        if len(token)>=min_length and p.match(token):
            filtered_tokens.append(token)
            
    return filtered_tokens

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Vectorization

In [ ]:
x= news.drop('label',axis=1)
x.head(2)

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...


In [ ]:
y = news['label']

In [ ]:
news.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [ ]:
news=news.dropna()

messeges =news.copy()
messeges.reset_index(inplace=True)
messeges.head()


,index,id,title,author,text,label
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
y=messeges['label']


In [ ]:
import nltk
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

ps = PorterStemmer()
nltk.download('stopwords')
corpus = []
for i in range(0, len(messeges)):
    review = re.sub('[^a-zA-Z]', ' ', messeges['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
cv = CountVectorizer(max_features=5000,ngram_range=(1,3))
X = cv.fit_transform(corpus).toarray()


In [ ]:
#from sklearn.feature_extraction.text import CountVectorizer
#bow = CountVectorizer(analyzer=cleanText).fit(news['text'])

In [ ]:
#matrix_bow = bow.transform(news['text'])
#print('sparse matrix shape:', matrix_bow.shape)
#print('total elements:',matrix_bow.shape[0] * matrix_bow.shape[1])
#print( 'number of non-zeros:', matrix_bow.nnz)
#print( 'sparsity: %.2f%%' % (100.0 * matrix_bow.nnz / (matrix_bow.shape[0] * matrix_bow.shape[1])))

TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
path2=sst_home+'fake-news/test'

train = news
test=pd.read_csv(path2)

test=test.fillna(' ')
train=train.fillna(' ')
test['total']=test['title']+' '+test['author']+test['text']
train['total']=train['title']+' '+train['author']+train['text']

#tfidf
transformer = TfidfTransformer(smooth_idf=False)
count_vectorizer = CountVectorizer(ngram_range=(1, 2))
counts = count_vectorizer.fit_transform(train['total'].values)
tfidf = transformer.fit_transform(counts)
targets = train['label'].values
test_counts = count_vectorizer.transform(test['total'].values)
test_tfidf = transformer.fit_transform(test_counts)

#split in samples
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(tfidf, targets, random_state=0, test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state =0, test_size=0.2)

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:1466: RuntimeWarning: divide by zero encountered in true_divide
  idf = np.log(n_samples / df) + 1


In [ ]:
 #ANTIGUO

from sklearn.feature_extraction.text import TfidfTransformer

#first, we train the transformer to tf-idf model
tfidf_transformer = TfidfTransformer().fit(matrix_bow)

#now, we transform the bow matrix to a new matrix based on tf-idf
tfidf_vectors = tfidf_transformer.transform(matrix_bow)

Training 

In [ ]:
#ANTIGUO

bow = CountVectorizer(analyzer=cleanText).fit(msg_train)
#transform the training set to bow model
bow_train = bow.transform(msg_train)
#transform the test set to bow model
bow_test=bow.transform(msg_test)


#learn the tf-idf model from the training bow
tfidf_transformer = TfidfTransformer().fit(bow_train)
#transform the training set to tf-idf model
tfidf_train = tfidf_transformer.transform(bow_train)
#transform the test set to tf-idf model
tfidf_test = tfidf_transformer.transform(bow_test)


Training and testing with a pipeline

In [ ]:
#ANTIGUO

from sklearn.model_selection import cross_val_score, train_test_split 
from sklearn.pipeline import Pipeline

#news_train, news_test, label_train, label_test = train_test_split(news['text'], news['label'], test_size=0.2)

pipeline = Pipeline([
    ('bow', CountVectorizer(analyzer=cleanText)),  # strings to token integer counts
    ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
    ('classifier', MultinomialNB()),  # train on TF-IDF vectors w/ Naive Bayes classifier
])

pipeline.fit(news_train,label_train)
predictions = pipeline.predict(news_test)

print( classification_report(label_test, predictions))


Multinomial Naive Bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB
classifier=MultinomialNB()
from sklearn import metrics
import numpy as np
import itertools
from sklearn.metrics import classification_report, confusion_matrix

classifier.fit(X_train, y_train)
pred = classifier.predict(X_test)
score = metrics.accuracy_score(y_test, pred)
print("accuracy:   %0.3f" % score)
#cm = metrics.confusion_matrix(y_test, pred)
print( classification_report(y_test, pred))

accuracy:   0.648
              precision    recall  f1-score   support

           0       0.61      1.00      0.76      2040
           1       1.00      0.20      0.34      1617

    accuracy                           0.65      3657
   macro avg       0.81      0.60      0.55      3657
weighted avg       0.78      0.65      0.57      3657



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
import graphviz
import scipy.stats as stats
#import optuna
import plotly
import math
from sklearn import tree, neighbors

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import PredefinedSplit
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer
#from skopt import BayesSearchCV
#from skopt.space import Integer, Real, Categorical
from scipy.stats import uniform, expon
from scipy.stats import randint as sp_randint
from numpy.random import randint
from sklearn.datasets import make_regression
from sklearn import ensemble
import xgboost as xgb
from sklearn.feature_selection import SelectKBest, f_regression, chi2

**SVM**


In [ ]:
from sklearn import svm

clf = svm.SVC()
clf.fit(X_train,y_train)

label_pred = clf.predict(X_test)

In [ ]:
print(confusion_matrix(y_test,label_pred))
print(classification_report(y_test,label_pred))

[[2001   39]
 [  92 1525]]
              precision    recall  f1-score   support

           0       0.96      0.98      0.97      2040
           1       0.98      0.94      0.96      1617

    accuracy                           0.96      3657
   macro avg       0.97      0.96      0.96      3657
weighted avg       0.96      0.96      0.96      3657



**Logistic** **regression**

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='liblinear', random_state=0).fit(X_train,y_train)

And evaluate the model

In [ ]:
model.predict_proba(X_test)

array([[0.58796507, 0.41203493],
       [0.69933089, 0.30066911],
       [0.02970106, 0.97029894],
       ...,
       [0.86746812, 0.13253188],
       [0.76584172, 0.23415828],
       [0.52702747, 0.47297253]])

In [ ]:
model.predict(X_test)

array([0, 0, 1, ..., 0, 0, 0])

In [ ]:
model.score(X_test,y_test)

0.9472245009570687

In [ ]:
confusion_matrix(y_test, model.predict(X_test))

array([[1977,   63],
       [ 130, 1487]])

In [ ]:
print(classification_report(y_test, model.predict(X_test)))

              precision    recall  f1-score   support

           0       0.94      0.97      0.95      2040
           1       0.96      0.92      0.94      1617

    accuracy                           0.95      3657
   macro avg       0.95      0.94      0.95      3657
weighted avg       0.95      0.95      0.95      3657



**KNN**

In [ ]:
#Convert categorical labels to binary ones

#import pandas as pd
#binary_test = pd.get_dummies(label_test)
#binary_test = binary_test.drop(["ham"], axis=1) 

#binary_train = pd.get_dummies(label_train)
#binary_train = binary_train.drop(["ham"], axis=1) 

np.random.seed(0)
neigh = KNeighborsClassifier()
start = time.time()
neigh = neigh.fit(X_train,y_train)
end = time.time()
totaltime_knn = end-start
label_train_pred = neigh.predict(X_train)
label_test_pred = neigh.predict(X_test)

final_model_tree = neigh.fit(X_train,y_train)


In [ ]:
print(classification_report(y_test, neigh.predict(X_test)))

              precision    recall  f1-score   support

           0       0.88      0.62      0.73      2040
           1       0.65      0.90      0.75      1617

    accuracy                           0.74      3657
   macro avg       0.77      0.76      0.74      3657
weighted avg       0.78      0.74      0.74      3657



**Random** **Forest**

In [ ]:
np.random.seed(0)
rf = RandomForestClassifier()
rf = rf.fit(X_train,y_train)
label_test_pred2 = rf.predict(X_test)

In [ ]:
print(classification_report(y_test, rf.predict(X_test)))

              precision    recall  f1-score   support

           0       0.86      0.98      0.91      2040
           1       0.97      0.79      0.87      1617

    accuracy                           0.90      3657
   macro avg       0.91      0.89      0.89      3657
weighted avg       0.91      0.90      0.90      3657



**Gradient** **Boosting**

In [ ]:
 np.random.seed(0)
gf = xgb.XGBClassifier()
gf = gf.fit(X_train,y_train)
label_test_pred3 = gf.predict(X_test)

In [ ]:
print(classification_report(y_test, gf.predict(X_test)))

              precision    recall  f1-score   support

           0       0.98      0.97      0.97      2040
           1       0.96      0.97      0.97      1617

    accuracy                           0.97      3657
   macro avg       0.97      0.97      0.97      3657
weighted avg       0.97      0.97      0.97      3657

